### 전처리된 음식점데이터 불러오기

In [ ]:
import pandas as pd
restaurant=pd.read_csv('test.csv',encoding='utf-8',error_bad_lines=False, low_memory = False)


In [ ]:
restaurant.info()

In [ ]:
restaurant.head()

### 아래에서 실행시켜서 입력

In [ ]:
xlsxname=input('파일명 입력 : ').strip() #같은경로에 있는 읽어올 엑셀 파일명 입력

In [ ]:
sheetlen=int(input('시트 개수 입력 : '))

In [ ]:
result_excel=pd.ExcelWriter(xlsxname+'_result.xlsx',engine='xlsxwriter')


In [ ]:
import sys

for stl in range(sheetlen):
    xlsxfile=pd.read_excel(xlsxname+'.xlsx',sheet_name=stl)
    xlsxfile['위도']=None
    xlsxfile['경도']=None
    xlsxfile['좌표정보(x)']=None
    xlsxfile['좌표정보(y)']=None
#     print(xlsxfile.columns)
    
    for idx, v in xlsxfile.iterrows():
        
        if '상호명' in xlsxfile.columns:
            name=str(v['상호명'])
        
        elif '업체명' in xlsxfile.columns:
            name=str(v['업체명'])
        
        elif '학교' in xlsxfile.columns:
            name=str(v['학교'])
        
        
        addr=v['주소']
       

        if '(' in addr:
            addr=' '.join(addr[:addr.index('(')].split()[1:])
        else:
            addr=' '.join(addr.split()[1:])


        #url 파트 
        a = list(addr.split())

        m = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query="

        for i in a:
            m += "+"
            m += i

        xlsxfile['네이버 Url'][idx] = m+"+"+name
        #####



    #     print(addr)
        try:
            if ('(' in name ) or (')' in name):
                tmp=restaurant[((restaurant['소재지전체주소'].str.contains(addr))|(restaurant['도로명전체주소'].str.contains(addr)))&(restaurant['사업장명']==name)]
            

            else:
                tmp=restaurant[((restaurant['소재지전체주소'].str.contains(addr))|(restaurant['도로명전체주소'].str.contains(addr)))&(restaurant['사업장명'].str.contains(name))]

            if len(tmp)>0:
                if tmp.head(1).reset_index()['상세영업상태명'][0]!='영업':
                    xlsxfile['영업 여부(1:Y/2:N)'][idx]=2

                else:
                    xlsxfile['영업 여부(1:Y/2:N)'][idx]=1

                    if xlsxfile['좌표정보(x)'][idx]==0 and xlsxfile['좌표정보(y)'][idx]==0:
                        continue
                    else:
                        xlsxfile['좌표정보(x)'][idx]=tmp.head(1).reset_index()['좌표정보(x)'][0]
                        xlsxfile['좌표정보(y)'][idx]=tmp.head(1).reset_index()['좌표정보(y)'][0]
                        xlsxfile['위도'][idx]=tmp.head(1).reset_index()['위도'][0]
                        xlsxfile['경도'][idx]=tmp.head(1).reset_index()['경도'][0]

                print('변경',idx)

            else:
                print('패스',idx)

                continue
        except:
            print('에러남 ')
            print(restaurant['소재지전체주소'])
            print(restaurant['도로명전체주소'])
            print(restaurant['사업장명'])
            print(addr)
            print(name)
            print(idx)
            sys.exit(0)

    xlsxfile=xlsxfile.reset_index(drop=True)
    
    xlsxfile.to_excel(result_excel,sheet_name=str(stl+1)+'번째 시트.xlsx', index=False)
    xlsxfile.to_csv(str(stl+1)+'번째 시트.csv', index=False)

result_excel.save()
# 엑셀저장